In [ ]:
# Prepare variables
import os
PROJECT_NAME = os.getenv("PROJECT_NAME", "demo_projectapi")
REFRESH_TOKEN = os.getenv("REFRESH_TOKEN")
CONSUMER_TOKEN = os.getenv("CONSUMER_TOKEN")
SENSITIVE_CONSUMER_TOKEN = os.getenv("SENSITIVE_CONSUMER_TOKEN")

In [ ]:
# PREP work - client initialization
from featurestore import *
client = Client(API, secure=True)
client.auth.set_auth_token(REFRESH_TOKEN)

In [ ]:
# Define credentials for ingesting user's data
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")
S3_REGION = os.getenv("S3_REGION")
credentials = S3Credentials(S3_ACCESS_KEY, S3_SECRET_KEY, S3_REGION)

In [ ]:
# project will be deleted if there exists already
try:
    client.projects.get(PROJECT_NAME).delete()
except:
    pass

## Project API

In [ ]:
# Create a project
project = client.projects.create(PROJECT_NAME)

In [ ]:
# List all the projects
client.projects.list()

In [ ]:
# Update project fields: description and locked
# Set locked to False; only users with owner and consumer access can view this project
project.locked = True
project.description = "Demo of Project API"

In [ ]:
# Specify input data source
source = CSVFile("s3a://feature-store-test-data/duplicate_match/combined_match.csv")

In [ ]:
# Extract the schema from datasource
schema = client.extract_schema_from_source(source, credentials)

In [ ]:
# Create a new feature set with the above schema
project.feature_sets.register(schema, "fs_project", "description")

In [ ]:
# Ingest data into the feature store
fs = project.feature_sets.get("fs_project")
fs.ingest(source, credentials)

In [ ]:
# Add "view only" access to consumer user 
project.add_consumers(["consumer@feature-store.ai"])

In [ ]:
# Logout the main_user
client.auth.logout()

In [ ]:
# Login as a consumer user
client.auth.set_auth_token(CONSUMER_TOKEN)

In [ ]:
# Consumer user is the active user
client.auth.get_active_user()

In [ ]:
# Consumer user can access the project and list feature sets created by the main_user
project = client.projects.get(PROJECT_NAME) 
[featureset for featureset in project.feature_sets.list()]

In [ ]:
# Logout the consumer user
client.auth.logout()

In [ ]:
# Login as sensitive_user
client.auth.set_auth_token(SENSITIVE_CONSUMER_TOKEN)

In [ ]:
# Should show "sensitive_user" as the active user
client.auth.get_active_user()

In [ ]:
# Sensitive_user cannot access the project and the feature sets created by the owner as sensitive_user doesn't have consumer permissions
project = client.projects.get(PROJECT_NAME)
try: 
    [featureset for featureset in project.feature_sets.list()]
except:
     print("This user does not have permission to view feature sets as the project is locked")

In [ ]:
# logout the sensitive_user
client.auth.logout()

## Cleanups

In [ ]:
client.auth.set_auth_token(REFRESH_TOKEN)
client.projects.get(PROJECT_NAME).delete()
